<a href="https://colab.research.google.com/github/siliconemonster/AD-TrabalhoFinal/blob/main/Simula%C3%A7%C3%A3o_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando bibliotecas

In [24]:
import random
from random import sample
from random import random
from matplotlib import pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
import math
from math import *
import numpy as np
from numpy import random
import pandas as pd
import queue 

In [25]:
def initializeCache():  
  cache = sample([1,2,3], k= 2)
  return cache

def cacheReceivesReqFIFO(cache, requisicao):
  RequicaoAtendida = 0

  if (requisicao in cache):
    cache = cache
    RequicaoAtendida = 1    
    print("Cache possui requisição!") 

  else:
    cache.insert(0, requisicao)
    del cache[-1]
    print("Novo estado da cache: " + str(cache)) 

  return cache, RequicaoAtendida

def cacheReceivesReq_LRU(cache, requisicao):
  RequicaoAtendida = 0

  if (requisicao in cache):
    RequicaoAtendida = 1    
    print("Cache possui requisição!")

    cache.remove(requisicao)
    cache.insert(0, requisicao)
    print("Novo estado da cache: " + str(cache)) 

  else:
    cache.insert(0, requisicao)
    del cache[-1]
    print("Novo estado da cache: " + str(cache)) 

  return cache, RequicaoAtendida


def cacheReceivesReqRandom(cache, requisicao):
  RequicaoAtendida = 0

  if (requisicao in cache):
    cache = cache
    RequicaoAtendida = 1    
    print("Cache possui requisção!")  

  else:
    cache.insert(0, requisicao)
    del cache[random.randint(-2,0)]
    print("Novo estado da cache: " + str(cache)) 

  return cache, RequicaoAtendida

In [26]:
def simulacao_FIFO(trials):
  queueCache1 = []
  queueCache2 = []
  sucesso = 0
  mediaSucesso = 0  
  cache1 = initializeCache()
  cache2 = initializeCache()
  
  for i in range (trials):
    RequicaoAtendida = 0
    requisicao = random.randint(1,4)
    print("Requisição: " + str(requisicao))
    print("Cache 1: " + str(cache1))
    print("Cache 2: " + str(cache2))

    cache1, RequicaoAtendida = cacheReceivesReqFIFO(cache1, requisicao)
    cache2, RequicaoAtendida = cacheReceivesReqFIFO(cache2, requisicao)

    if(RequicaoAtendida):
      sucesso = sucesso + 1

  mediaSucesso = sucesso/trials 
  print("Media Sucessos =" + str(mediaSucesso))
  return mediaSucesso


def simulacao_LRU(trials):
  queueCache1 = []
  queueCache2 = []
  sucesso = 0
  mediaSucesso = 0  
  cache1 = initializeCache()
  cache2 = initializeCache()
  
  for i in range (trials):
    RequicaoAtendida = 0
    requisicao = random.randint(1,4)
    print("Requisição: " + str(requisicao))
    print("Cache 1: " + str(cache1))
    print("Cache 2: " + str(cache2))

    cache1, RequicaoAtendida = cacheReceivesReq_LRU(cache1, requisicao)
    cache2, RequicaoAtendida = cacheReceivesReq_LRU(cache2, requisicao)

    if(RequicaoAtendida):
      sucesso = sucesso + 1

  mediaSucesso = sucesso/trials 
  print("Media Sucessos =" + str(mediaSucesso))
  return mediaSucesso


def simulacao_RANDOM(trials):
  queueCache1 = []
  queueCache2 = []
  sucesso = 0
  mediaSucesso = 0  
  cache1 = initializeCache()
  cache2 = initializeCache()
  
  for i in range (trials):
    RequicaoAtendida = 0
    requisicao = random.randint(1,4)
    print("Requisição: " + str(requisicao))
    print("Cache 1: " + str(cache1))
    print("Cache 2: " + str(cache2))

    cache1, RequicaoAtendida = cacheReceivesReqRandom(cache1, requisicao)
    cache2, RequicaoAtendida = cacheReceivesReqRandom(cache2, requisicao)

    if(RequicaoAtendida):
      sucesso = sucesso + 1

  mediaSucesso = sucesso/trials 
  print("Media Sucessos =" + str(mediaSucesso))
  return mediaSucesso
     

In [27]:
simulacao_LRU(10)

Requisição: 2
Cache 1: [3, 2]
Cache 2: [2, 1]
Cache possui requisição!
Novo estado da cache: [2, 3]
Cache possui requisição!
Novo estado da cache: [2, 1]
Requisição: 2
Cache 1: [2, 3]
Cache 2: [2, 1]
Cache possui requisição!
Novo estado da cache: [2, 3]
Cache possui requisição!
Novo estado da cache: [2, 1]
Requisição: 2
Cache 1: [2, 3]
Cache 2: [2, 1]
Cache possui requisição!
Novo estado da cache: [2, 3]
Cache possui requisição!
Novo estado da cache: [2, 1]
Requisição: 3
Cache 1: [2, 3]
Cache 2: [2, 1]
Cache possui requisição!
Novo estado da cache: [3, 2]
Novo estado da cache: [3, 2]
Requisição: 2
Cache 1: [3, 2]
Cache 2: [3, 2]
Cache possui requisição!
Novo estado da cache: [2, 3]
Cache possui requisição!
Novo estado da cache: [2, 3]
Requisição: 2
Cache 1: [2, 3]
Cache 2: [2, 3]
Cache possui requisição!
Novo estado da cache: [2, 3]
Cache possui requisição!
Novo estado da cache: [2, 3]
Requisição: 2
Cache 1: [2, 3]
Cache 2: [2, 3]
Cache possui requisição!
Novo estado da cache: [2, 3]
C

0.8